# LSTM model for observation operator

## Load in the required data

In [ ]:
import os
from pathlib import Path
pad = Path(os.getcwd())
if pad.name == "ml_observation_operator":
    pad_correct = pad.parent
    os.chdir(pad_correct)
%run "ml_observation_operator/data_load_in.py"

## Load in used packages

In [ ]:
# import geopandas as gpd
# import pandas as pd
import numpy as np
import xarray as xr
from numba import njit
import matplotlib.pyplot as plt
import hvplot
import hvplot.pandas
import hvplot.xarray
import tensorflow as tf
from typing import Tuple

print(os.getcwd())
%load_ext autoreload 
%autoreload 2 

Check the distance between the days

In [ ]:
deltat_t_arr = np.zeros((features.shape[0],))
for i in range(features.shape[0]-1):
    dt_temp = features.index[i+1] - features.index[i]
    deltat_t_arr[i] = dt_temp.total_seconds()/(3600*24)
deltat_t_arr[i+1] = np.nan
features['deltat_t'] = deltat_t_arr
display(features.head())
features['deltat_t'].hvplot(frame_width = 1000)


In [ ]:
features['VVPasture'].hvplot()

In [ ]:
s1_xr_g0 = xr.open_dataset('data/g0_OpenEO/g0_zwalm_landuse.nc')
display(s1_xr_g0)
s1_xr_g0['VV_db'] = 10*np.log10(s1_xr_g0['VV'])
s1_xr_g0['VV_db'].hvplot.image('x','y', geo = True, crs = 32631, 
                            tiles = 'OSM', cmap = 'bwr', frame_width = 320,
                            rasterize = True)

Not really regular timeseries...

https://stats.stackexchange.com/questions/312609/rnn-for-irregular-time-intervals

PDM related

# Reshape the data

Function used from Frederik Kratzert in https://github.com/kratzert/pangeo_lstm_example/blob/master/LSTM_for_rainfall_runoff_modelling.ipynb

In [ ]:
@njit
def reshape_data(x: np.ndarray, y: np.ndarray, seq_length: int) -> Tuple[np.ndarray, np.ndarray]:
    """
    Reshape matrix data into sample shape for LSTM training.

    :param x: Matrix containing input features column wise and time steps row wise
    :param y: Matrix containing the output feature.
    :param seq_length: Length of look back days for one day of prediction
    
    :return: Two np.ndarrays, the first of shape (samples, length of sequence,
        number of features), containing the input data for the LSTM. The second
        of shape (samples, 1) containing the expected output for each input
        sample.
    """
    num_samples, num_features = x.shape

    x_new = np.zeros((num_samples - seq_length + 1, seq_length, num_features))
    y_new = np.zeros((num_samples - seq_length + 1, 1))

    for i in range(0, x_new.shape[0]):
        x_new[i, :, :num_features] = x[i:i + seq_length, :]
        y_new[i, :] = y[i + seq_length - 1, 0]

    return x_new, y_new

In [ ]:
X_train_np = X_train.values
y_train_np = y_train.values.reshape(-1,1)
X_test_np = X_test.values
y_test_np = y_test.values.reshape(-1,1)
print(X_train_np.shape)
print(y_train_np.shape)

X_train_lstm, y_train_lstm = reshape_data(X_train_np, y_train_np, seq_length=60)
print(X_train_lstm.shape)
print(y_train_lstm.shape)

So logical that you reduce from 579 to 520 training samples as yo can only start predicting form day 1!

In [ ]:
X_train.values.shape
y_train.values.shape

In [ ]:
type(X_train.values)

## LSTM model

In [ ]:
import tensorflow as tf